# **1-Recherche du dataset**

je vais utiliser deux modules de python:

**-requests** : je vais l'utiliser pour envoyer la requette https get afin d'obtenir la page html qui contient notre texte.

**-bs4** :  ce module va me permettre d'extraire le texte contenue dans la page html.

j'ai choisi une page web qui contient un texte qui long afin qu'il soit suffisant pour le diviser en 2 , une partie pour l'apprentissage et une autre pour le test .

In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import random
import itertools
import string
html=requests.get("https://whiletrue.neocities.org/lte.html")
html=html.text
clean_text=''.join(BeautifulSoup(html,"html.parser").stripped_strings)
lower_case_text=clean_text.lower()

# 2-Prétraitement du texte

dans cette partie j'ai éliminé les poctuations qui existe dans le texte ainsi que d'autre caractères comme : ð , ¾ comme j'ai aussi remplacé les caractères qui ont un accent circonflexe par des caractères sans accent.

In [ ]:
punctuations = '''ð¾º°\x82½!()¼\x83-—³\r\n\x88\x92[]\x9c{}©\x81“”;\x80:¸\x9d'’"\,'‘'<>±.µ\x99/?@+#$%^&*_~\x981234567890\x84²'''
no_punct_text = ""
for char in lower_case_text:
   if char not in punctuations:
       no_punct_text = no_punct_text + char
no_punct_text=no_punct_text.replace('â','a')
no_punct_text=no_punct_text.replace('ñ','n')

#3-Division du texte en 2 parties (apprentissage et test)

In [ ]:
train_text=no_punct_text[:67700]
test_text=no_punct_text[67700:-1]

#4-chaine de markov d'ordre 1

dans cette étape je vais construire la matrice des probabilités de transition.

In [ ]:
# retourne l'indice du caractère dans la matrice des probabilités
chars=[chr(i) for i in range(97,123)]
chars.append(' ')
def get_char_index(char):
  return chars.index(char)
# construction de la matrice des probabilités:
def markov_chain_order_1(train_text):
  # la matrice des transitions
  M=np.array(np.zeros((27,27)))
  for x in range(len(train_text)-1):
    i=get_char_index(train_text[x])
    j=get_char_index(train_text[x+1])
    M[i][j]+=1
  # division de chaque ligne par la somme des élements de cette ligne pour qu'il soit une probabilité
  sum=np.sum(M,axis=1)
  for i in range(len(M)):
      if sum[i]!=0:
          M[i]=M[i]/sum[i]
  return M

#5-évaluation du modèle de modèle de Markov

In [ ]:
def model_evaluation(test_text,M):
  score=0
  for x in range(len(test_text)-1):
    sc=0
    i=get_char_index(test_text[x])
    j=get_char_index(test_text[x+1])
    arr=[]
    arr=M[i].tolist()
    ind=chars.index(np.random.choice(chars,p=M[i]))
    tentative=1
    sc+=M[i][j]
    while(chars[ind]!=chars[j]):
        tentative+=1
        arr[ind]=-1
        ind = chars.index(np.random.choice(chars,p=M[i]))
    sc=sc/tentative
    score+=sc
  return score


In [ ]:
M=markov_chain_order_1(train_text)
score=model_evaluation(test_text,M)
print("score : ",score)

score :  33.69616661068976


l'évaluation du modèle de markov nous a donné un score de 34.51

#6-modèle de markov d'ordre supérieur (d'ordre n)

on va suivre les memes étapes qu'auparavant sauf que maintenant les états de la chaines ne sont plus des caractères , elles sont des mots.

d'abord on doit créer un tableau qui contient les combinaisons de n caractères afin de faciliter l'obtention de l'indice d'une combinaison:

je vais mettre tout d'abord n=3

In [ ]:

def char_combination(n):
  strs=[]
  for item in itertools.product(chars, repeat=n):
    string=''.join(item)
    strs.append(string)
  return strs
def get_str_index(string,strs):
  return strs.index(string)


In [ ]:
#construction de la matrice des transitions:
def markov_chain_order_n(train_text,n):
  strs=char_combination(n)
  Mn=np.array(np.zeros((27**n,27)))
  for x in range(len(train_text)-n):
    i=get_str_index(train_text[x:x+n],strs)
    j=get_char_index(train_text[x+n])
    Mn[i][j] += 1
  # division de chaque ligne par la somme des élements de cette ligne pour qu'il soit une probabilité
  sum=np.sum(Mn,axis=1)
  for i in range(len(Mn)):
      if sum[i]!=0:
          Mn[i]=Mn[i]/sum[i]
  return Mn

#7-évaluation du modèle:

In [ ]:
def evaluate_markov_model(M,test_text,n):
  score=0
  strs=char_combination(n)
  for x in range(len(test_text)-n):
    sc=0
    i=get_str_index(test_text[x:x+n],strs)
    j=get_char_index(test_text[x+n])
    arr=[]
    arr=M[i].tolist()
    ind = arr.index(max(arr))
    tentative=1
    sc+=M[i][j]
    while(chars[ind]!=chars[j]):
        tentative+=1
        arr[ind]=-1
        ind = arr.index(max(arr))
    sc=sc/tentative
    score+=sc
  return score

In [ ]:
M=markov_chain_order_n(train_text,n=2)
score=evaluate_markov_model(M,test_text,n=2)
print("score : ",score)

score :  112.17816972080074


l'évaluation du modèle de markov d'ordre 3 nous a donnée un score de 112.17 qui est très grand du score donné par le modèle d'ordre 1.

# 8-Modélisation du texte comme succession des mots

dans cette section les états sont les mots du texte.

In [ ]:
def get_words_of_text(train_text):
  words=train_text.split()
  #pour avoir des mots uniques:
  words=list(dict.fromkeys(words))
  return words
def get_word_combination(n,words):
  strs=[]
  for item in itertools.product(words, repeat=n):
    string=''.join(item)
    strs.append(string)
  return strs
def get_word_index(words_combination,word):
  return words_combination.index(word)
def get_words_index(words_combination,words):
  return words_combination.index(words)


In [ ]:
def words_markov_model_order_n(train_text,n=1):
  words=get_words_of_text(train_text)
  spaced_words=[]
  for i in range(len(words)):
    spaced_words.append(words[i]+" ")
  length=len(words)
  if n!=1:
    words_combination=get_word_combination(n,spaced_words)
  else:
    words_combination=words
  M=np.zeros((length**n,length))
  empty=" "
  splitted_text=train_text.split()
  for x in range(len(splitted_text)-n):
    i=get_words_index(words_combination,empty.join(splitted_text[x:x+n]))
    j=get_word_index(words,splitted_text[x+n])
    M[i][j]+=1
  # division de chaque ligne par la somme des élements de cette ligne pour qu'il soit une probabilité
  sum=np.sum(M,axis=1)
  for i in range(len(M)):
      if sum[i]!=0:
          M[i]=M[i]/sum[i]
  return M

# 9-Géneration du texte avec un modèle de markov

In [ ]:
def text_generation(M,length,start,n):
  strs=char_combination(n)
  text=start
  string=start
  for i in range(length):
    index=get_str_index(string,strs)
    prob=M[index]
    try:
      char_next=np.random.choice(chars,p=prob)
    except(ValueError):
      char_next=np.random.choice(chars)
    text=text+char_next
    string=text[i+1:]
  return text



In [ ]:
n_max=4
Ms=[]
# pour avoir les matrices de transition des 5 modèles:
for i in range(1,n_max+1):
  Ms.append(markov_chain_order_n(train_text,i))


In [ ]:
start="i can"
length=200
for i in range(1,n_max+1):
  text=text_generation(Ms[i-1],length,start[:i],i)
  print("----------------the text generated by the model of order :",i,"----------------")
  print(text)

----------------the text generated by the model of order : 1 ----------------
itins heatisighonat whe lit thanguelsed bla  g a dasa os sthe ar mong tifo g tyont odanore ho tthe giting ractsto juem s ung fly me thine myinase barean t be we fit dag urle ing eadn tell i futheng wr 
----------------the text generated by the model of order : 2 ----------------
i som sheret days a smake my i lted a durecookmadere the evessic it rit meher ter hat this this spefers tharks up my mone thoulat sent funding only dre beticiall ter of causeembit hatime vied yof yout e
----------------the text generated by the model of order : 3 ----------------
i chardent howed the typing abouture top editiesame i dont toast just on i works still of differse ill as i maths technice not they havent pastill king and closite yournerpost stand them prevery good it 
----------------the text generated by the model of order : 4 ----------------
i can made cringy store i dont know what anothere cring my dad so this eative a

In [ ]:
def words_text_generation(M,length,start,n=1):
  word=start
  text=start
  for i in range(length):
    words=get_words_of_text(train_text)
    combinations=get_word_combination(n,words)
    index=get_word_index(combinations,word)
    prob=M[index]
    next_word=np.random.choice(combinations,p=prob)
    text+=" "+next_word
    word=next_word
  return text

In [ ]:
M=words_markov_model_order_n(train_text)
start="the"
length=200
text=words_text_generation(M,length,start)
print(text)

the days when it at home from a sesame street piano is reduced to her lte except the t its being a furry by talking about now my part of them in this lte its a countdown on the lte is way to have to write a world record three months ago i was yet but we take a bomb thats quite nice too boring part of people are on there and try to happen okay i didnat know ill be an important message theres not why because i see if my language is this webpage has an hour plus the cyrillic you knew i need a language my sister sometimes but you dont think i did a story once but i mean i once made by the exclamation point but its on my sister is even if i believe was a users computerconsolewhatever session is write as much as possible in this lte out as i dont think of the start to zula the country according to increase your ltes flamingchicken lte fact it was in this language to get from march and dont know one copy because of this be the fact you never finished it properly but i


# 10-Comparaison et conclusions

le texte géneré par le modèle de markov d'ordre n est meilleur et bien compréhensible que celui géneré par les modèles d'ordre n-1,n-2,...,1 et il a un score plus grand .malheureusement on peut pas aller vers un ordre qui est supérieur à 4 car on est limité par la taille de la mémoire et aussi on a la taille de la matrice des transitions qui grandit avec l'ordre donc ce qui permet à la phase d'apprentissage de prendre beaucoup de temps et meme parfois la session peut se bloquer.

la version des mots est la meilleure puisque un modèle d'ordre 1 génére des mots qui sont compréhensible mais meme ici malheuresement on ne peut pas dépasser n=1, si l'on dépasse ca va prendre beacoups de temps et la session se bloque vers la fin.